# Practical Exam: Hotel Operations

LuxurStay Hotels is a major, international chain of hotels. They offer hotels for both business and leisure travellers in major cities across the world. The chain prides themselves on the level of customer service that they offer. 

However, the management has been receiving complaints about slow room service in some hotel branches. As these complaints are impacting the customer satisfaction rates, it has become a serious issue. Recent data shows that customer satisfaction has dropped from the 4.5 rating that they expect. 

You are working with the Head of Operations to identify possible causes and hotel branches with the worst problems. 

## Data

The following schema diagram shows the tables available. You have only been provided with data where customers provided a feedback rating.

![hotel_operations](hotel_operations.png)

# Task 1

Before you can start any analysis, you need to confirm that the data is accurate and reflects what you expect to see. 

It is known that there are some issues with the `branch` table, and the data team have provided the following data description. 

Write a query to return data matching this description, including identifying and cleaning all invalid values. You must match all column names and description criteria. Your output should be a DataFrame named 'clean_branch_data'.

| Column Name | Criteria                                                |
|-------------|---------------------------------------------------------|
|id | Nominal. The unique identifier of the hotel. </br>Missing values are not possible due to the database structure.|
| location | Nominal. The location of the particular hotel. One of four possible values, 'EMEA', 'NA', 'LATAM' and 'APAC'. </br>Missing values should be replaced with “Unknown”. |
| total_rooms | Discrete. The total number of rooms in the hotel. Must be a positive integer between 1 and 400. </br>Missing values should be replaced with the default number of rooms, 100. |
| staff_count | Discrete. The number of staff employeed in the hotel service department. </br>Missing values should be replaced with the total_rooms multiplied by 1.5. |
| opening_date | Discrete. The year in which the hotel opened. This can be any value between 2000 and 2023. </br>Missing values should be replaced with 2023. |
| target_guests | Nominal. The primary type of guest that is expected to use the hotel. Can be one of 'Leisure' or 'Business'. </br>Missing values should be replaced with 'Leisure'. |

In [23]:
SELECT
    id,
    
    -- Clean location: Standardize valid values, replace others with 'Unknown'
    COALESCE(
        CASE 
            WHEN UPPER(TRIM(location)) IN ('EMEA', 'NA', 'LATAM', 'APAC') 
            THEN UPPER(TRIM(location))
            ELSE NULL
        END,
        'Unknown'
    ) AS location,
    
    -- Clean total_rooms: Ensure between 1-400, default to 100
    COALESCE(
        CASE 
            WHEN total_rooms BETWEEN 1 AND 400 THEN total_rooms
            ELSE NULL
        END,
        100
    ) AS total_rooms,
    
    -- Clean staff_count: Replace invalid with total_rooms*15 (or 1500 if total_rooms invalid)
    COALESCE(
        CASE 
            WHEN staff_count > 0 THEN staff_count
            ELSE NULL
        END,
        (COALESCE(
            CASE 
                WHEN total_rooms BETWEEN 1 AND 400 THEN total_rooms
                ELSE NULL
            END,
            100
        )) * 15
    ) AS staff_count,
    
    -- Clean opening_date: Validate year range, default to 2023
    COALESCE(
        CASE 
            WHEN opening_date ~ '^\-?\d+$' AND opening_date::integer BETWEEN 2000 AND 2023 
            THEN opening_date::integer
            ELSE NULL
        END,
        2023
    ) AS opening_date,
    
    -- Clean target_guests: Standardize to 'Business' or 'Leisure'
    COALESCE(
        CASE 
            WHEN UPPER(TRIM(target_guests)) LIKE 'B%' THEN 'Business'
            WHEN UPPER(TRIM(target_guests)) LIKE 'L%' THEN 'Leisure'
            ELSE NULL
        END,
        'Leisure'
    ) AS target_guests
FROM branch;

,id,location,total_rooms,staff_count,opening_date,target_guests
0,1,LATAM,168,178,2017,Business
1,2,APAC,154,82,2010,Leisure
2,3,APAC,212,467,2003,Leisure
3,4,APAC,230,387,2023,Business
4,5,APAC,292,293,2002,Business
...,...,...,...,...,...,...
95,96,APAC,237,257,2000,Business
96,97,APAC,107,169,2005,Business
97,98,EMEA,196,126,2002,Leisure
98,99,APAC,242,251,2021,Business


# Task 2

The Head of Operations wants to know whether there is a difference in time taken to respond to a customer request in each hotel. They already know that different services take different lengths of time. 

Calculate the average and maximum duration for each branch and service. 
- Your output should be a DataFrame named 'average_time_service'
- It should include the columns `service_id`, `branch_id`, `avg_time_taken` and `max_time_taken`
- Values should be rounded to two decimal places where appropriate. 

In [24]:
SELECT
    service_id,
    branch_id,
    ROUND(AVG(time_taken)::numeric, 2) AS avg_time_taken,
    ROUND(MAX(time_taken)::numeric, 2) AS max_time_taken
FROM request
GROUP BY service_id, branch_id
ORDER BY service_id, branch_id;


,service_id,branch_id,avg_time_taken,max_time_taken
0,1,1,2.44,12.0
1,1,2,2.14,6.0
2,1,3,2.28,6.0
3,1,4,2.18,9.0
4,1,5,2.52,10.0
...,...,...,...,...
380,4,96,9.79,26.0
381,4,97,9.59,21.0
382,4,98,9.14,14.0
383,4,99,9.13,13.0


# Task 3

The management team want to target improvements in `Meal` and `Laundry` service in Europe (`EMEA`) and Latin America (`LATAM`). 

Write a query to return the `description` of the service, the `id` and `location` of the branch, the id of the request as `request_id` and the `rating` for the services and locations of interest to the management team. 

Your output should be a DataFrame named 'target_hotels'.

Use the original branch table, not the output of task 1. 

In [25]:
SELECT
    s.description,
    b.id AS branch_id,
    b.location,
    r.id AS request_id,
    r.rating
FROM request r
JOIN service s ON r.service_id = s.id
JOIN branch b ON r.branch_id = b.id
WHERE s.description IN ('Meal', 'Laundry')
  AND b.location IN ('EMEA', 'LATAM')
ORDER BY b.location, s.description, r.rating DESC;

,description,branch_id,location,request_id,rating
0,Laundry,82,EMEA,15585,5
1,Laundry,62,EMEA,3921,5
2,Laundry,79,EMEA,13679,5
3,Laundry,40,EMEA,1969,5
4,Laundry,66,EMEA,15583,5
...,...,...,...,...,...
5042,Meal,1,LATAM,13737,3
5043,Meal,60,LATAM,13738,3
5044,Meal,1,LATAM,10971,3
5045,Meal,64,LATAM,13754,3


# Task 4

So that you can take a more detailed look at the lowest performing hotels, you want to get service and branch information where the average rating for the branch and service combination is lower than 4.5 - the target set by management.  

- Your output should be a DataFrame named 'average_rating'
- It should return the `service_id` and `branch_id`, and the average rating (`avg_rating`)
- Values should be rounded to 2 decimal places where appropriate.

In [26]:
SELECT
    service_id,
    branch_id,
    ROUND(AVG(rating)::numeric, 2) AS avg_rating
FROM request
GROUP BY service_id, branch_id
HAVING AVG(rating) < 4.5
ORDER BY avg_rating ASC;


,service_id,branch_id,avg_rating
0,4,20,3.00
1,4,39,3.00
2,2,100,3.00
3,2,99,3.29
4,3,8,3.38
...,...,...,...
210,4,24,4.00
211,4,34,4.33
212,1,90,4.38
213,4,54,4.40
